In [1]:
pip install pandas sqlalchemy


In [4]:
pip install mysql-connector-python


     --------------------------------------- 14.2/14.2 MB 13.9 MB/s eta 0:00:00
     ------------------------------------- 527.0/527.0 kB 11.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [66]:
import pandas as pd
from sqlalchemy import create_engine

# Ruta al archivo Excel (xlsx)
excel_path = r'C:\Users\USUARIO\seguridadsocial2\Libro1.xlsx'

# Configuración de la conexión a la base de datos MySQL
db_config = {
    'host': 'localhost',
    'user': 'root', 
    'port': 3306, 
    'password': '', 
    'database':'pensiones' 
}

# Crear una conexión a la base de datos MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# Leer el archivo Excel en un DataFrame
df = pd.read_excel(excel_path)

# Definir el nombre de la tabla en la base de datos
table_name = 'datos'  

# Cargar los datos en la tabla de la base de datos y reemplazar si ya existen
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f'Datos cargados en la tabla "{table_name}".')


Datos cargados en la tabla "datos".


In [70]:
import pandas as pd
from sqlalchemy import create_engine

# Ruta al archivo Excel (xlsx)
excel_path = r'C:\Users\USUARIO\seguridadsocial2\Libro1.xlsx'

# Configuración de la conexión a la base de datos MySQL
db_config = {
    'host': 'localhost',
    'user': 'root',
    'port': 3306,
    'password': '',
    'database': 'pensiones'
}

# Crear una conexión a la base de datos MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# Leer el archivo Excel en un DataFrame
df = pd.read_excel(excel_path)

# Definir el nombre de la tabla en la base de datos
table_name = 'datos'

# Obtener una lista de Números de Identificación (asumiendo que la columna en el DataFrame se llama 'Numero_Identificacion')
existing_ids = pd.read_sql(f"SELECT Numero_Identificacion FROM {table_name}", engine)['Numero_Identificacion'].tolist()

# Filtrar filas que ya existen en la base de datos
existing_rows = df[df['Numero_Identificacion'].isin(existing_ids)]

# Filtrar filas que no existen en la base de datos
new_rows = df[~df['Numero_Identificacion'].isin(existing_ids)]

# Actualizar las filas existentes en la base de datos
existing_rows.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Agregar las nuevas filas a la base de datos
new_rows.to_sql(table_name, con=engine, if_exists='append', index=False)

print(f'Datos actualizados en la tabla "{table_name}".')



Datos actualizados en la tabla "datos".


In [71]:
import pandas as pd
from sqlalchemy import create_engine

# Ruta al archivo Excel (xlsx)
excel_path = r'C:\Users\USUARIO\seguridadsocial2\TablaPensionistasParaMsql.xlsx'

# Configuración de la conexión a la base de datos MySQL
db_config = {
    'host': 'localhost',
    'user': 'root',
    'port': 3306,
    'password': '',
    'database': 'pensiones'
}

# Crear una conexión a la base de datos MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# Leer el archivo Excel en un DataFrame
df = pd.read_excel(excel_path)

# Definir el nombre de la tabla en la base de datos
table_name = 'datos'

# Agregar nuevas filas y actualizar las existentes
df.to_sql(table_name, con=engine, if_exists='append', index=False)

print(f'Datos cargados en la tabla "{table_name}".')


Datos cargados en la tabla "datos".


In [7]:
import pandas as pd
from sqlalchemy import create_engine, Column, String, DateTime, Float, Integer, Boolean, BigInteger
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

# Ruta al archivo Excel (xlsx)
excel_path = input("Por favor, ingrese la ruta del archivo Excel: ")
# Configuración de la conexión a la base de datos MySQL
db_config = {
    'host': 'localhost',
    'user': 'root',
    'port': 3306,
    'password': '',
    'database': 'pensiones'
}

# Crear una conexión a la base de datos MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

Base = declarative_base()

# Define la estructura de la tabla
class Datos(Base):
    __tablename__ = 'datos'
    Nombre = Column(String(collation='utf8mb4_general_ci'))
    Apellidos = Column(String(collation='utf8mb4_general_ci'))
    Numero_Identificacion = Column(String(100), primary_key=True)
    Fecha_de_Nacimiento = Column(DateTime)
    Fecha_de_Jubilacion = Column(DateTime)
    Bases_de_Cotizacion = Column(Float)
    Comunidad_Autonoma = Column(String(collation='utf8mb4_general_ci'))
    Invalidez = Column(Boolean)
    Base_Reguladora = Column(Float)
    Total_Meses_Cotizados = Column(Integer)
    Pension = Column(Float)
    Pension_Final = Column(Float)
    Anos_Enteros_Cotizados = Column(Integer)
    Meses_Cotizados = Column(Integer)
    Anos_Cotizados_Decimal = Column(Float)
    Edad = Column(Integer)
    Grupo_Edades = Column(String(collation='utf8mb4_general_ci'))
    Pension_Final_con_Complementos = Column(Float)

# Crear la tabla si no existe
Base.metadata.create_all(engine)

# Crear una sesión de SQLAlchemy
Session = sessionmaker(bind=engine)
session = Session()

# Leer el archivo Excel en un DataFrame
df = pd.read_excel(excel_path)

# Itera sobre las filas del DataFrame y agrega o actualiza los datos en la base de datos
for index, row in df.iterrows():
    dato = Datos(
        Nombre=row['Nombre'],
        Apellidos=row['Apellidos'],
        Numero_Identificacion=str(row['Numero_Identificacion']),
        Fecha_de_Nacimiento=row['Fecha_de_Nacimiento'],
        Fecha_de_Jubilacion=row['Fecha_de_Jubilacion'],
        Bases_de_Cotizacion=row['Bases_de_Cotizacion'],
        Comunidad_Autonoma=row['Comunidad_Autonoma'],
        Invalidez=bool(row['Invalidez']),
        Base_Reguladora=row['Base_Reguladora'],
        Total_Meses_Cotizados=row['Total_Meses_Cotizados'],
        Pension=row['Pension'],
        Pension_Final=row['Pension_Final'],
        Anos_Enteros_Cotizados=row['Anos_Enteros_Cotizados'],
        Meses_Cotizados=row['Meses_Cotizados'],
        Anos_Cotizados_Decimal=row['Anos_Cotizados_Decimal'],
        Edad=row['Edad'],
        Grupo_Edades=row['Grupo_Edades'],
        Pension_Final_con_Complementos=row['Pension_Final_con_Complementos']
    )
    session.merge(dato)

# Intenta guardar los cambios en la base de datos
try:
    session.commit()
    print('Datos cargados en la tabla "datos".')
except IntegrityError as e:
    session.rollback()
    print('Error al cargar los datos:', str(e))

# Cierra la sesión
session.close()


Datos cargados en la tabla "datos".


In [13]:
import pandas as pd
from sqlalchemy import create_engine, Column, String, DateTime, Float, Integer, Boolean, BigInteger
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

# Ruta al archivo Excel (xlsx)
excel_path = input("Por favor, ingrese la ruta del archivo Excel: ")

# Configuración de la conexión a la base de datos MySQL
db_config = {
    'host': 'localhost',
    'user': 'root',
    'port': 3306,
    'password': '',
    'database': 'pensiones'
}

# Crear una conexión a la base de datos MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

Base = declarative_base()

# Define la estructura de la tabla
class Datos(Base):
    __tablename__ = 'datos'
    Nombre = Column(String(collation='utf8mb4_general_ci'))
    Apellidos = Column(String(collation='utf8mb4_general_ci'))
    Numero_Identificacion = Column(String(100), primary_key=True)
    Fecha_de_Nacimiento = Column(DateTime)
    Fecha_de_Jubilacion = Column(DateTime)
    Bases_de_Cotizacion = Column(Float)
    Comunidad_Autonoma = Column(String(collation='utf8mb4_general_ci'))
    Invalidez = Column(Boolean)
    Base_Reguladora = Column(Float)
    Total_Meses_Cotizados = Column(Integer)
    Pension = Column(Float)
    Pension_Final = Column(Float)
    Anos_Enteros_Cotizados = Column(Integer)
    Meses_Cotizados = Column(Integer)
    Anos_Cotizados_Decimal = Column(Float)
    Edad = Column(Integer)
    Grupo_Edades = Column(String(collation='utf8mb4_general_ci'))
    Pension_Final_con_Complementos = Column(Float)

# Crear la tabla si no existe
Base.metadata.create_all(engine)

# Crear una sesión de SQLAlchemy
Session = sessionmaker(bind=engine)
session = Session()

# Leer el archivo Excel en un DataFrame
df = pd.read_excel(excel_path)

# DataFrame para registros erróneos
registros_erroneos = pd.DataFrame(columns=df.columns)

# Itera sobre las filas del DataFrame y agrega o actualiza los datos en la base de datos
for index, row in df.iterrows():
    # Comprobar si alguno de los campos está vacío
    if row.isnull().any():
        registros_erroneos = registros_erroneos.append(row)
    else:
        try:
            dato = Datos(
                Nombre=row['Nombre'],
                Apellidos=row['Apellidos'],
                Numero_Identificacion=str(row['Numero_Identificacion']),
                Fecha_de_Nacimiento=row['Fecha_de_Nacimiento'],
                Fecha_de_Jubilacion=row['Fecha_de_Jubilacion'],
                Bases_de_Cotizacion=row['Bases_de_Cotizacion'],
                Comunidad_Autonoma=row['Comunidad_Autonoma'],
                Invalidez=bool(row['Invalidez']),
                Base_Reguladora=row['Base_Reguladora'],
                Total_Meses_Cotizados=row['Total_Meses_Cotizados'],
                Pension=row['Pension'],
                Pension_Final=row['Pension_Final'],
                Anos_Enteros_Cotizados=row['Anos_Enteros_Cotizados'],
                Meses_Cotizados=row['Meses_Cotizados'],
                Anos_Cotizados_Decimal=row['Anos_Cotizados_Decimal'],
                Edad=row['Edad'],
                Grupo_Edades=row['Grupo_Edades'],
                Pension_Final_con_Complementos=row['Pension_Final_con_Complementos']
            )
            session.merge(dato)
        except Exception as e:
            registros_erroneos = registros_erroneos.append(row)

# Intenta guardar los cambios en la base de datos
try:
    session.commit()
    print('Datos válidos cargados en la tabla "datos".')
except IntegrityError as e:
    session.rollback()
    print('Error al cargar los datos válidos:', str(e))

# Guarda los registros erróneos en un archivo Excel
if not registros_erroneos.empty:
    registros_erroneos.to_excel('Registros_Erroneos.xlsx', index=False)
    print('Registros erróneos guardados en "Registros_Erroneos.xlsx".')


Datos válidos cargados en la tabla "datos".
Registros erróneos guardados en "Registros_Erroneos.xlsx".


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12172\3231096072.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  registros_erroneos = registros_erroneos.append(row)
